<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/XGBoost_undersample_submission_1v30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#경로 설정 
import os
os.chdir('/content/drive/MyDrive/A2W/데이터/final')

In [3]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression



from xgboost import plot_importance
from xgboost import XGBClassifier

삭제해야할 변수
수종그룹코드(KOFTR_TROUP_CD)
-	81,82,83, 91, 92, 93, 94, 95, 96, 99
-	 


# 데이터 처리 및 탐색

In [4]:
# Train 데이터셋
raw_df = pd.read_csv("sum_all_realfinal.csv", encoding='utf-8-sig')
#raw_df = pd.read_csv("merge_final_real1.csv", encoding='utf-8-sig')

# Test 검증 데이터셋
test_input = pd.read_csv('test_final.csv', encoding = 'utf-8-sig')
#불필요한 칼럼제거
X_test = test_input.drop(['1day_yn','2day_yn','date','sd','sgg','umd_x','1day','2day','tm','stnId','stnNm','umd_y','umd2'], axis =1)

In [ ]:
raw_df

In [ ]:
raw_df['PRRCK_LARG'].value_counts()

2.0    162475
1.0    154223
3.0     78489
Name: PRRCK_LARG, dtype: int64

In [ ]:
clean_df = raw_df.replace(to_replace = np.nan, value = 0)

In [ ]:
clean_df = raw_df.fillna(0)

In [7]:
raw_df1 = raw_df[['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'umd2']]
raw_df2 = raw_df[['PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT', 'landslide',
       '1day_yn', '2day_yn']]

In [10]:
raw_df2 = raw_df2.fillna(0)

In [ ]:
clean_df.columns

Index(['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'umd2', 'PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT', 'landslide',
       '1day_yn', '2day_yn'],
      dtype='object')

In [ ]:
clean_df.info()

NameError: ignored

In [ ]:
clean_df = clean_df.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

In [14]:
clean_df = pd.concat([raw_df1, raw_df2], axis=1)
clean_df

,stnId,stnNm,tm,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,umd2,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,landslide,1day_yn,2day_yn
0,115,울릉도,2011-06-01,13.6,12.7,14.9,0.0,0.5,6.67,2.0,10.6,7.0,3.1,81.0,95.5,1.6,울릉군 울릉읍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0,0.0,0.0
1,115,울릉도,2011-06-02,13.4,11.8,16.5,0.0,0.1,0.67,0.1,5.8,3.7,1.3,74.0,92.5,NaN,울릉군 울릉읍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0,0.0,0.0
2,115,울릉도,2011-06-03,14.6,12.0,18.7,NaN,NaN,NaN,NaN,13.3,8.5,5.9,48.0,88.1,NaN,울릉군 울릉읍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0,0.0,0.0
3,115,울릉도,2011-06-04,20.7,18.2,23.9,NaN,NaN,NaN,NaN,19.2,12.0,6.2,29.0,53.4,NaN,울릉군 울릉읍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0,0.0,0.0
4,115,울릉도,2011-06-05,21.0,17.9,23.6,NaN,NaN,NaN,NaN,13.1,9.2,3.6,37.0,48.0,NaN,울릉군 울릉읍,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402067,295,남해,2019-10-27,12.1,8.3,18.0,NaN,NaN,NaN,NaN,3.6,1.7,1.0,33.0,61.6,NaN,남해군 삼동면,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0,0.0,0.0
402068,295,남해,2019-10-28,13.4,7.3,19.0,NaN,NaN,NaN,NaN,4.7,2.9,1.0,45.0,65.6,NaN,남해군 삼동면,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0,0.0,0.0
402069,295,남해,2019-10-29,15.4,10.7,21.0,NaN,NaN,NaN,NaN,6.4,4.1,1.6,35.0,62.6,NaN,남해군 삼동면,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0,0.0,0.0
402070,295,남해,2019-10-30,13.3,9.0,20.0,NaN,NaN,NaN,NaN,3.7,1.8,1.1,40.0,64.6,NaN,남해군 삼동면,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0,0.0,0.0


In [15]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402072 entries, 0 to 402071
Data columns (total 39 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   stnId       402072 non-null  int64  
 1   stnNm       402072 non-null  object 
 2   tm          402072 non-null  object 
 3   avgTa       401578 non-null  float64
 4   minTa       402002 non-null  float64
 5   maxTa       401966 non-null  float64
 6   mi10MaxRn   167072 non-null  float64
 7   hr1MaxRn    167127 non-null  float64
 8   sumRnDur    33601 non-null   float64
 9   sumRn       167814 non-null  float64
 10  maxInsWs    401976 non-null  float64
 11  maxWs       401976 non-null  float64
 12  avgWs       401882 non-null  float64
 13  minRhm      401934 non-null  float64
 14  avgRhm      401356 non-null  float64
 15  n99Rn       28552 non-null   float64
 16  umd2        402072 non-null  object 
 17  PRRCK_LARG  402072 non-null  float64
 18  PRRCK_MDDL  402072 non-null  float64
 19  LO

In [ ]:
pd.options.display.float_format = '{:.5f}'.format
clean_df.describe()

,stnId,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,landslide,1day_yn,2day_yn
count,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000,402072.00000
mean,245.19762,21.83967,17.57915,27.24464,0.76801,2.12924,0.50835,5.62295,6.66098,3.81126,1.50239,51.49321,75.92667,0.95117,1.77739,20.24310,318.14794,26.12975,174.28151,2.83907,10.66092,2.55480,1.55137,15.86989,2.14039,4.50666,0.99658,1.89384,1.51370,24.00693,1.71919,4.02399,12.27401,0.00089,0.00089,0.00089
std,55.49525,4.91446,5.55439,4.90126,2.27176,5.97367,2.37791,18.12925,2.57062,1.53913,0.86327,17.24170,11.58660,7.97039,0.77313,7.73672,189.44734,8.37611,100.89731,0.96767,2.74204,2.09870,0.80714,8.12726,1.27308,6.31717,0.05842,0.31896,0.62194,17.26778,0.58219,0.96307,4.21107,0.02987,0.02987,0.02987
min,115.00000,0.00000,-7.10000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,192.00000,19.00000,14.50000,23.80000,0.00000,0.00000,0.00000,0.00000,5.00000,2.80000,0.90000,39.00000,68.90000,0.00000,1.00000,12.00000,167.80000,20.30000,89.50000,2.00000,12.00000,1.00000,1.00000,10.00000,1.00000,1.00000,1.00000,2.00000,1.00000,11.00000,2.00000,4.00000,10.00000,0.00000,0.00000,0.00000
50%,277.00000,22.40000,18.50000,27.50000,0.00000,0.00000,0.00000,0.00000,6.30000,3.60000,1.30000,50.00000,76.40000,0.00000,2.00000,22.00000,292.00000,25.30000,170.10000,3.00000,12.00000,1.00000,1.00000,10.00000,2.00000,2.00000,1.00000,2.00000,1.00000,15.00000,2.00000,4.00000,12.00000,0.00000,0.00000,0.00000
75%,284.00000,25.40000,21.80000,30.80000,0.00000,0.90000,0.00000,1.10000,7.80000,4.50000,1.80000,63.00000,83.80000,0.00000,2.00000,25.00000,426.40000,30.90000,270.40000,4.00000,12.00000,5.00000,2.00000,20.00000,3.00000,2.00000,1.00000,2.00000,2.00000,34.00000,2.00000,5.00000,14.00000,0.00000,0.00000,0.00000
max,295.00000,34.10000,29.30000,40.40000,29.00000,119.50000,24.00000,332.90000,39.40000,26.00000,14.90000,100.00000,100.00000,318.70000,3.00000,31.00000,883.80000,72.10000,358.90000,4.00000,12.00000,6.00000,3.00000,30.00000,9.00000,28.00000,1.00000,2.00000,3.00000,77.00000,3.00000,5.00000,22.00000,1.00000,1.00000,1.00000


In [ ]:
# if adjust_ratio: # pulsar 데이터 증폭 flow
#     data = np.zeros([2 * star_cnt, 9])
#     data[0:star_cnt,:] = np.asarray(stars, dtype = 'float32')

#     for n in range(star_cnt):
#         data[star_cnt + n] = np.asarray(pulsars[n % pulsar_cnt], dtype = 'float32')

# else:
#     data = np.zeros([star_cnt + pulsar_cnt, 9])
#     data[0:star_cnt,:] = np.asarray(stars, dtype = 'float32')
#     data[star_cnt:,:] = np.asarray(pulsars, dtype = 'float32')

# 이상치 탐지

In [ ]:
#특정 행들만 추출
#columns = X_test.columns.tolist()
#columns.append("landslide")
#columns1 = columns.copy()
#day = ['1day_yn','2day_yn']
#columns1.extend(day)
#df = clean_df[columns]
#columns1
#df_all = clean_df[columns1]

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
%matplotlib inline

model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1,
                      max_features=2, contamination=0.01)
model.fit(data_df_scaled.to_numpy())

NameError: ignored

In [ ]:
score = model.decision_function(data_df_scaled.to_numpy())
anomaly = model.predict(data_df_scaled.to_numpy())
data_df_scaled['scores']= score
data_df_scaled['anomaly']= anomaly
anomaly_data = data_df_scaled.loc[data_df_scaled['anomaly']==-1] # 이상값은 -1으로 나타낸다.
anomaly_data

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,LOCTN_ALTT,LOCTN_GRDN,stnId,PRRCK_LARG_0.0,PRRCK_LARG_2.0,PRRCK_LARG_3.0,PRRCK_LARG_1.0,PRRCK_MDDL_0.0,PRRCK_MDDL_23.0,PRRCK_MDDL_22.0,PRRCK_MDDL_31.0,PRRCK_MDDL_11.0,PRRCK_MDDL_24.0,PRRCK_MDDL_21.0,PRRCK_MDDL_12.0,PRRCK_MDDL_32.0,PRRCK_MDDL_26.0,PRRCK_MDDL_15.0,PRRCK_MDDL_14.0,PRRCK_MDDL_16.0,PRRCK_MDDL_25.0,EIGHT_AGL_dir_N,EIGHT_AGL_dir_W,EIGHT_AGL_dir_E,EIGHT_AGL_dir_S,CLZN_CD_0.0,CLZN_CD_2.0,...,KOFTR_GROU_82.0,KOFTR_GROU_14.0,KOFTR_GROU_15.0,KOFTR_GROU_0.0,KOFTR_GROU_44.0,KOFTR_GROU_33.0,KOFTR_GROU_36.0,KOFTR_GROU_49.0,KOFTR_GROU_17.0,KOFTR_GROU_78.0,KOFTR_GROU_16.0,KOFTR_GROU_31.0,KOFTR_GROU_12.0,DMCLS_CD_1.0,DMCLS_CD_2.0,DMCLS_CD_3.0,DMCLS_CD_0.0,AGCLS_CD_5.0,AGCLS_CD_6.0,AGCLS_CD_4.0,AGCLS_CD_1.0,AGCLS_CD_3.0,AGCLS_CD_0.0,AGCLS_CD_2.0,AGCLS_CD_7.0,AGCLS_CD_9.0,HEIGHT_16.0,HEIGHT_12.0,HEIGHT_14.0,HEIGHT_18.0,HEIGHT_10.0,HEIGHT_0.0,HEIGHT_8.0,HEIGHT_20.0,HEIGHT_4.0,HEIGHT_6.0,HEIGHT_22.0,HEIGHT_24.0,scores,anomaly
0,-1.31880,-0.93325,-1.78721,-0.23632,-0.25605,8.33811,1.89829,7.49650,8.59866,9.66591,1.26164,1.00814,0.89286,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.09751,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00590,-1
1,-1.39159,-0.91370,-1.87018,-0.23632,-0.25605,9.88513,1.65074,2.40299,2.47635,4.41607,1.26164,1.53286,5.03064,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.09751,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00491,-1
2,-1.43318,-0.96258,-1.92204,-0.23632,-0.25605,5.46508,0.31241,2.88157,3.46758,4.93076,1.05890,0.87988,-0.05934,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.09751,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00153,-1
5,-1.64114,-1.15808,-1.95315,-0.23632,-0.25605,9.81441,2.41660,3.39434,4.22558,4.31313,0.29861,1.67861,4.68438,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.09751,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00562,-1
8,-1.55796,-1.23628,-1.77684,-0.23632,-0.25605,8.93040,1.62753,4.11222,5.68328,3.79844,0.60272,1.22969,3.52442,-1.42174,-2.06220,-2.12502,4.94769,-0.83

In [ ]:
#label_df.info()
data_df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093790 entries, 0 to 2093789
Columns: 145 entries, avgTa to anomaly
dtypes: float64(144), int64(1)
memory usage: 2.3 GB


In [ ]:
scaled_df_all = pd.concat([data_df_scaled,label_df],axis=1)
scaled_df_all

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,LOCTN_ALTT,LOCTN_GRDN,stnId,PRRCK_LARG_0.0,PRRCK_LARG_2.0,PRRCK_LARG_3.0,PRRCK_LARG_1.0,PRRCK_MDDL_0.0,PRRCK_MDDL_23.0,PRRCK_MDDL_22.0,PRRCK_MDDL_31.0,PRRCK_MDDL_11.0,PRRCK_MDDL_24.0,PRRCK_MDDL_21.0,PRRCK_MDDL_12.0,PRRCK_MDDL_32.0,PRRCK_MDDL_26.0,PRRCK_MDDL_15.0,PRRCK_MDDL_14.0,PRRCK_MDDL_16.0,PRRCK_MDDL_25.0,EIGHT_AGL_dir_N,EIGHT_AGL_dir_W,EIGHT_AGL_dir_E,EIGHT_AGL_dir_S,CLZN_CD_0.0,CLZN_CD_2.0,...,KOFTR_GROU_0.0,KOFTR_GROU_44.0,KOFTR_GROU_33.0,KOFTR_GROU_36.0,KOFTR_GROU_49.0,KOFTR_GROU_17.0,KOFTR_GROU_78.0,KOFTR_GROU_16.0,KOFTR_GROU_31.0,KOFTR_GROU_12.0,DMCLS_CD_1.0,DMCLS_CD_2.0,DMCLS_CD_3.0,DMCLS_CD_0.0,AGCLS_CD_5.0,AGCLS_CD_6.0,AGCLS_CD_4.0,AGCLS_CD_1.0,AGCLS_CD_3.0,AGCLS_CD_0.0,AGCLS_CD_2.0,AGCLS_CD_7.0,AGCLS_CD_9.0,HEIGHT_16.0,HEIGHT_12.0,HEIGHT_14.0,HEIGHT_18.0,HEIGHT_10.0,HEIGHT_0.0,HEIGHT_8.0,HEIGHT_20.0,HEIGHT_4.0,HEIGHT_6.0,HEIGHT_22.0,HEIGHT_24.0,scores,anomaly,landslide,1day_yn,2day_yn
0,-1.31880,-0.93325,-1.78721,-0.23632,-0.25605,8.33811,1.89829,7.49650,8.59866,9.66591,1.26164,1.00814,0.89286,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00590,-1,0.00000,0.00000,0.00000
1,-1.39159,-0.91370,-1.87018,-0.23632,-0.25605,9.88513,1.65074,2.40299,2.47635,4.41607,1.26164,1.53286,5.03064,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00491,-1,0.00000,0.00000,0.00000
2,-1.43318,-0.96258,-1.92204,-0.23632,-0.25605,5.46508,0.31241,2.88157,3.46758,4.93076,1.05890,0.87988,-0.05934,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,-0.00153,-1,0.00000,0.00000,0.00000
3,-1.22522,-0.78662,-1.60053,-0.23632,-0.25605,-0.20585,-0.26778,1.27489,1.48512,1.43086,0.65341,-0.12291,-0.11128,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.01597,1,0.00000,0.00000,0.00000
4,-1.27721,-0.94303,-1.73536,-0.23632,-0.25605,2.99869,-0.12854,2.19788,1.54343,2.46024,0.09587,-0.21619,2.01820,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,

In [ ]:
clean_all = scaled_df_all.drop(anomaly_data.index)
clean_all.head()

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,LOCTN_ALTT,LOCTN_GRDN,stnId,PRRCK_LARG_0.0,PRRCK_LARG_2.0,PRRCK_LARG_3.0,PRRCK_LARG_1.0,PRRCK_MDDL_0.0,PRRCK_MDDL_23.0,PRRCK_MDDL_22.0,PRRCK_MDDL_31.0,PRRCK_MDDL_11.0,PRRCK_MDDL_24.0,PRRCK_MDDL_21.0,PRRCK_MDDL_12.0,PRRCK_MDDL_32.0,PRRCK_MDDL_26.0,PRRCK_MDDL_15.0,PRRCK_MDDL_14.0,PRRCK_MDDL_16.0,PRRCK_MDDL_25.0,EIGHT_AGL_dir_N,EIGHT_AGL_dir_W,EIGHT_AGL_dir_E,EIGHT_AGL_dir_S,CLZN_CD_0.0,CLZN_CD_2.0,...,KOFTR_GROU_0.0,KOFTR_GROU_44.0,KOFTR_GROU_33.0,KOFTR_GROU_36.0,KOFTR_GROU_49.0,KOFTR_GROU_17.0,KOFTR_GROU_78.0,KOFTR_GROU_16.0,KOFTR_GROU_31.0,KOFTR_GROU_12.0,DMCLS_CD_1.0,DMCLS_CD_2.0,DMCLS_CD_3.0,DMCLS_CD_0.0,AGCLS_CD_5.0,AGCLS_CD_6.0,AGCLS_CD_4.0,AGCLS_CD_1.0,AGCLS_CD_3.0,AGCLS_CD_0.0,AGCLS_CD_2.0,AGCLS_CD_7.0,AGCLS_CD_9.0,HEIGHT_16.0,HEIGHT_12.0,HEIGHT_14.0,HEIGHT_18.0,HEIGHT_10.0,HEIGHT_0.0,HEIGHT_8.0,HEIGHT_20.0,HEIGHT_4.0,HEIGHT_6.0,HEIGHT_22.0,HEIGHT_24.0,scores,anomaly,landslide,1day_yn,2day_yn
3,-1.22522,-0.78662,-1.60053,-0.23632,-0.25605,-0.20585,-0.26778,1.27489,1.48512,1.43086,0.65341,-0.12291,-0.11128,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.01597,1,0.00000,0.00000,0.00000
4,-1.27721,-0.94303,-1.73536,-0.23632,-0.25605,2.99869,-0.12854,2.19788,1.54343,2.46024,0.09587,-0.21619,2.01820,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.00170,1,0.00000,0.00000,0.00000
6,-1.44358,-1.08965,-1.70424,-0.23632,-0.25605,5.46508,0.04939,1.54837,1.42681,0.71029,0.09587,0.19775,-0.00741,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.00303,1,0.00000,0.00000,0.00000
7,-1.15243,-0.81595,-1.44496,-0.23632,-0.25605,-0.20585,-0.26778,1.99277,1.95158,1.63674,-0.36031,-0.75257,0.87555,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.01044,1,0.00000,0.00000,0.00000
9,-1.57875,-1.20696,-1.87018,-0.23632,-0.25605,6.60987,-0.05118,1.03560,1.31020,0.91617,0.14655,-0.19870,0.07916,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,

In [ ]:
clean_all.drop(['scores','anomaly'],axis=1)

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,LOCTN_ALTT,LOCTN_GRDN,stnId,PRRCK_LARG_0.0,PRRCK_LARG_2.0,PRRCK_LARG_3.0,PRRCK_LARG_1.0,PRRCK_MDDL_0.0,PRRCK_MDDL_23.0,PRRCK_MDDL_22.0,PRRCK_MDDL_31.0,PRRCK_MDDL_11.0,PRRCK_MDDL_24.0,PRRCK_MDDL_21.0,PRRCK_MDDL_12.0,PRRCK_MDDL_32.0,PRRCK_MDDL_26.0,PRRCK_MDDL_15.0,PRRCK_MDDL_14.0,PRRCK_MDDL_16.0,PRRCK_MDDL_25.0,EIGHT_AGL_dir_N,EIGHT_AGL_dir_W,EIGHT_AGL_dir_E,EIGHT_AGL_dir_S,CLZN_CD_0.0,CLZN_CD_2.0,...,KOFTR_GROU_14.0,KOFTR_GROU_15.0,KOFTR_GROU_0.0,KOFTR_GROU_44.0,KOFTR_GROU_33.0,KOFTR_GROU_36.0,KOFTR_GROU_49.0,KOFTR_GROU_17.0,KOFTR_GROU_78.0,KOFTR_GROU_16.0,KOFTR_GROU_31.0,KOFTR_GROU_12.0,DMCLS_CD_1.0,DMCLS_CD_2.0,DMCLS_CD_3.0,DMCLS_CD_0.0,AGCLS_CD_5.0,AGCLS_CD_6.0,AGCLS_CD_4.0,AGCLS_CD_1.0,AGCLS_CD_3.0,AGCLS_CD_0.0,AGCLS_CD_2.0,AGCLS_CD_7.0,AGCLS_CD_9.0,HEIGHT_16.0,HEIGHT_12.0,HEIGHT_14.0,HEIGHT_18.0,HEIGHT_10.0,HEIGHT_0.0,HEIGHT_8.0,HEIGHT_20.0,HEIGHT_4.0,HEIGHT_6.0,HEIGHT_22.0,HEIGHT_24.0,landslide,1day_yn,2day_yn
3,-1.22522,-0.78662,-1.60053,-0.23632,-0.25605,-0.20585,-0.26778,1.27489,1.48512,1.43086,0.65341,-0.12291,-0.11128,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.00000,0.00000,0.00000
4,-1.27721,-0.94303,-1.73536,-0.23632,-0.25605,2.99869,-0.12854,2.19788,1.54343,2.46024,0.09587,-0.21619,2.01820,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.00000,0.00000,0.00000
6,-1.44358,-1.08965,-1.70424,-0.23632,-0.25605,5.46508,0.04939,1.54837,1.42681,0.71029,0.09587,0.19775,-0.00741,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.00000,0.00000,0.00000
7,-1.15243,-0.81595,-1.44496,-0.23632,-0.25605,-0.20585,-0.26778,1.99277,1.95158,1.63674,-0.36031,-0.75257,0.87555,-1.42174,-2.06220,-2.12502,4.94769,-0.83315,-0.46188,-0.77492,4.94769,-0.20211,-0.39507,-0.43919,-0.52831,-0.41086,-0.06879,-0.16559,-0.11971,-0.19355,-0.03965,-0.33129,-0.17534,-0.22999,1.65081,-0.58883,-0.58400,-0.53074,4.94769,-0.76201,...,-0.12629,-0.38706,-0.13857,-0.05612,-0.06879,-0.05612,-0.06879,-0.09751,-0.03965,-0.03965,-0.07949,-0.05612,2.31732,-1.72662,-0.15529,-0.27571,1.62481,-0.14991,-1.09041,-0.16052,-0.26228,-0.21840,-0.14991,-0.03965,-0.03965,2.41857,-0.60576,-0.58400,-0.22231,-0.41346,-0.31950,-0.12629,-0.13255,-0.05612,-0.03965,-0.07949,-0.03965,0.00000,0.00000,0.00000
9,-1.57875,-1.20696,-1.87018,-0.23632,-0.25605,6.60987,-0.05118,1.03560,1.31020,0.91617,0.14655,-0.19870,0.07916,-1.42174,-2.06220,-2.

# 1일 후 예측

#### 변수 선택

In [ ]:
clean_df.head()

,stnId,stnNm,tm,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,umd2,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,landslide,1day_yn,2day_yn
0,115,울릉도,2011-06-01,13.60000,12.70000,14.90000,0.00000,0.50000,6.67000,2.00000,10.60000,7.00000,3.10000,81.00000,95.50000,1.60000,울릉군 울릉읍,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,2.00000,2.00000,30.00000,1.00000,5.00000,16.00000,0.00000,0.00000,0.00000
1,115,울릉도,2011-06-02,13.40000,11.80000,16.50000,0.00000,0.10000,0.67000,0.10000,5.80000,3.70000,1.30000,74.00000,92.50000,0.00000,울릉군 울릉읍,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,2.00000,2.00000,30.00000,1.00000,5.00000,16.00000,0.00000,0.00000,0.00000
2,115,울릉도,2011-06-03,14.60000,12.00000,18.70000,0.00000,0.00000,0.00000,0.00000,13.30000,8.50000,5.90000,48.00000,88.10000,0.00000,울릉군 울릉읍,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,2.00000,2.00000,30.00000,1.00000,5.00000,16.00000,0.00000,0.00000,0.00000
3,115,울릉도,2011-06-04,20.70000,18.20000,23.90000,0.00000,0.00000,0.00000,0.00000,19.20000,12.00000,6.20000,29.00000,53.40000,0.00000,울릉군 울릉읍,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,2.00000,2.00000,30.00000,1.00000,5.00000,16.00000,0.00000,0.00000,0.00000
4,115,울릉도,2011-06-05,21.00000,17.90000,23.60000,0.00000,0.00000,0.00000,0.00000,13.10000,9.20000,3.60000,37.00000,48.00000,0.00000,울릉군 울릉읍,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,2.00000,2.00000,30.00000,1.00000,5.00000,16.00000,0.00000,0.00000,0.00000


In [ ]:
#xgboostd의 feature importance를 통해 선정된 변수
columns_select_day1= ['avgRhm', 'maxTa','minTa','maxInsWs','LOCTN_ALTT','maxWs','avgTa','LOCTN_GRDN','avgWs','minRhm','sumRn','mi10MaxRn','hr1MaxRn','sumRnDur','CLZN_CD_4.0','HEIGHT_14.0','AGCLS_CD_5.0','PRRCK_MDDL_22.0','n99Rn','AGCLS_CD_4.0','PRRCK_MDDL_11.0','KOFTR_GROU_34.0','EIGHT_AGL_dir_S','HEIGHT_8.0','EIGHT_AGL_dir_N','KOFTR_GROU_30.0'] 

In [16]:
#xgboostd의 feature importance를 통해 선정된 변수
columns_select_day1= ['avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT'] 

In [17]:
clean_df.columns

Index(['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'umd2', 'PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT', 'landslide',
       '1day_yn', '2day_yn'],
      dtype='object')

In [18]:
#y = raw_df['1day_yn']
#X = raw_df[columns_select_day1]

y = clean_df['1day_yn']
X = clean_df[columns_select_day1]
#X = clean_df.drop(['1day_yn','2day_yn','tm','umd2','stnId'], axis =1)

In [19]:
X = pd.DataFrame(X, columns=columns_select_day1)
y = pd.DataFrame(y, columns = ['1day_yn'])
df_1day = pd.concat([X,y],axis=1)
df_1day

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,1day_yn
0,13.6,12.7,14.9,0.0,0.5,6.67,2.0,10.6,7.0,3.1,81.0,95.5,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
1,13.4,11.8,16.5,0.0,0.1,0.67,0.1,5.8,3.7,1.3,74.0,92.5,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
2,14.6,12.0,18.7,NaN,NaN,NaN,NaN,13.3,8.5,5.9,48.0,88.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
3,20.7,18.2,23.9,NaN,NaN,NaN,NaN,19.2,12.0,6.2,29.0,53.4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
4,21.0,17.9,23.6,NaN,NaN,NaN,NaN,13.1,9.2,3.6,37.0,48.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402067,12.1,8.3,18.0,NaN,NaN,NaN,NaN,3.6,1.7,1.0,33.0,61.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0
402068,13.4,7.3,19.0,NaN,NaN,NaN,NaN,4.7,2.9,1.0,45.0,65.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0
402069,15.4,10.7,21.0,NaN,NaN,NaN,NaN,6.4,4.1,1.6,35.0,62.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0
402070,13.3,9.0,20.0,NaN,NaN,NaN,NaN,3.7,1.8,1.1,40.0,64.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0


In [20]:
day0 = df_1day[df_1day['1day_yn'] == 0].shape[0]
day1 = df_1day[df_1day['1day_yn'] == 1].shape[0]

In [21]:
df1_1y = df_1day[df_1day['1day_yn'] == 1].copy()
df1_1y

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,1day_yn
892,16.1,14.6,19.1,0.0,0.8,6.20,0.9,11.5,7.4,4.1,63.0,75.8,0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,1.0
2726,18.3,16.7,20.1,0.0,0.7,4.25,0.9,22.8,11.4,5.5,69.0,80.1,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,1.0
4103,20.3,16.6,24.8,NaN,NaN,NaN,NaN,7.3,4.7,2.2,67.0,83.6,NaN,2.0,22.0,626.9,36.7,123.9,2.0,12.0,1.0,1.0,10.0,3.0,13.0,1.0,1.0,1.0,12.0,1.0,3.0,14.0,1.0
5480,20.3,16.6,24.8,NaN,NaN,NaN,NaN,7.3,4.7,2.2,67.0,83.6,NaN,2.0,23.0,583.3,38.7,279.9,2.0,12.0,5.0,3.0,20.0,4.0,14.0,1.0,1.0,1.0,11.0,0.0,1.0,0.0,1.0
6857,20.3,16.6,24.8,NaN,NaN,NaN,NaN,7.3,4.7,2.2,67.0,83.6,NaN,1.0,11.0,100.0,15.5,50.8,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,1.0,11.0,1.0,4.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396826,21.6,19.3,25.9,NaN,NaN,NaN,NaN,6.3,3.6,2.2,48.0,67.5,NaN,1.0,11.0,501.3,30.4,30.3,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,1.0,15.0,2.0,5.0,12.0,1.0
398203,21.6,19.3,25.9,NaN,NaN,NaN,NaN,6.3,3.6,2.2,48.0,67.5,NaN,1.0,11.0,501.3,30.4,30.3,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,1.0,15.0,2.0,5.0,12.0,1.0
399580,21.6,19.3,25.9,NaN,NaN,NaN,NaN,6.3,3.6,2.2,48.0,67.5,NaN,1.0,11.0,334.9,25.8,118.1,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,2.0,34.0,2.0,4.0,8.0,1.0
400210,20.8,17.0,27.3,NaN,NaN,NaN,NaN,3.6,1.8,1.0,32.0,75.1,NaN,1.0,11.0,334.9,25.8,118.1,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,2.0,34.0,2.0,4.0,8.0,1.0


1118

In [22]:
for _ in range(day0//day1):
  df_1day = pd.concat([df_1day,df1_1y],axis=0)
  df_1day

In [23]:
df_1day[df_1day['1day_yn'] == 0].shape

(401713, 33)

In [24]:
df_1day[df_1day['1day_yn'] == 1].shape

(401721, 33)

In [25]:
X_1by1 = df_1day.drop(['1day_yn'],axis=1).reset_index(drop=True)
y_1by1 = df_1day['1day_yn'].reset_index(drop=True)

# SAMPLING

In [ ]:
https://shinminyong.tistory.com/34

# ADASYN(Adaptive Synthetic Sampling) 

In [ ]:
from imblearn.combine import ADASYN

X_samp, y_samp = ADASYN(random_state=0).fit_sample(X, y)



ImportError: ignored

In [ ]:
from imblearn.combine import SMOTETomek

X_samp, y_samp = SMOTETomek(random_state=4).fit_sample(X, y)

plt.subplot(121)
classification_result2(X, y)
plt.subplot(122)
model_samp = classification_result2(X_samp, y_samp)

ValueError: ignored

# 오버 샘플링

In [ ]:
X = X.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

In [ ]:
#SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_over, y_train_over = smote.fit_sample(X, y)
X_train_over = pd.DataFrame(X_train_over, columns=columns_select_day1)
y_train_over = pd.DataFrame(y_train_over)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train_over

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT
0,13.600000,12.700000,14.900000,0.0,0.500000,6.670000,2.000000,10.600000,7.000000,3.100000,81.000000,95.500000,1.6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.0,2.000000,30.000000,1.00000,5.000000,16.000000
1,13.400000,11.800000,16.500000,0.0,0.100000,0.670000,0.100000,5.800000,3.700000,1.300000,74.000000,92.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.0,2.000000,30.000000,1.00000,5.000000,16.000000
2,14.600000,12.000000,18.700000,0.0,0.000000,0.000000,0.000000,13.300000,8.500000,5.900000,48.000000,88.100000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.0,2.000000,30.000000,1.00000,5.000000,16.000000
3,20.700000,18.200000,23.900000,0.0,0.000000,0.000000,0.000000,19.200000,12.000000,6.200000,29.000000,53.400000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.0,2.000000,30.000000,1.00000,5.000000,16.000000
4,21.000000,17.900000,23.600000,0.0,0.000000,0.000000,0.000000,13.100000,9.200000,3.600000,37.000000,48.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.0,2.000000,30.000000,1.00000,5.000000,16.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803421,20.504960,16.819957,24.964968,0.0,0.029994,0.000000,0.029994,8.039854,5.259890,2.344971,66.600079,83.320055,0.0,1.000000,11.149970,101.484708,15.504999,49.390278,4.000000,11.750049,1.000000,1.000000,10.000000,2.000000,1.000000,1.0,2.0,1.000000,11.199961,1.04999,4.000000,8.199961
803422,19.385272,15.463968,26.352445,0.0,0.000000,0.054103,0.000000,5.219674,4.370489,1.098370,62.590219,92.639237,0.0,2.000000,23.295109,308.547832,30.208262,218.931252,2.098370,12.000000,1.000000,1.000000,10.000000,3.000000,13.196740,1.0,2.0,1.000000,11.000000,2.00000,4.098370,10.393479
803423,20.810051,16.677653,26.520663,0.0,0.000000,0.000000,0.000000,6.078776,2.945817,1.455867,42.782144,73.457552,0.0,1.000000,11.000000,413.755361,22.519595,202.120718,2.776530,12.000000,6.000000,3.000000,28.882651,1.888265,2.000000,1.0,2.0,1.111735,13.122963,2.00000,4.111735,11.776530
803424,20.827387,18.196268,25.596474,0.0,0.000000,0.000000,0.000000,6.575933,3.903526,1.979254,52.690862,73.680901,0.0,1.275933,16.759331,260.181956,21.097915,241.978842,3.724067,10.620335,2.103732,1.275933,15.518662,2.275933,5.138996,1.0,2.0,1.551866,32.107851,2.00000,4.275933,12.000000


### 언더샘플링

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)
#X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 7180}).fit_sample(X, y)
X_samp = pd.DataFrame(X_samp, columns=columns_select_day1)
y_samp = pd.DataFrame(y_samp)

#df_samp = pd.concat([X_df, y_df], axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_samp = X_samp.astype('float')
X_samp = X_samp.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

In [26]:
#사용할 모듈 불러오기
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
#metrics 함수 정의
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

### Train, test 분리

In [27]:
# 1:1 ratio
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1by1, y_1by1, test_size=0.25, shuffle=True, stratify=y_1by1, random_state=1024)

#normal case
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=0.25, shuffle=True, stratify=y, random_state=1024)

# under sampling 분리
# 훈련 데이터와 테스트 데이터를 0.75:0.25의 비율로 분할
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_samp, y_samp, test_size=0.25, shuffle=True, stratify=y_samp, random_state=0)

#over sampliung 분리
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_train_over, y_train_over, test_size=0.25, shuffle=True, stratify=y_train_over, random_state=0)

print('X_train shape:', X_train_1.shape)
print('X_test shape:', X_test_1.shape)
print('y_train shape:', y_train_1.shape)
print('y_test shape:', y_test_1.shape)

X_train shape: (602575, 32)
X_test shape: (200859, 32)
y_train shape: (602575,)
y_test shape: (200859,)


# CATBOOST

In [28]:
pip install catboost

     |████████████████████████████████| 67.4 MB 29 kB/s 


In [29]:
X_train_1 = X_train_1.astype({'PRRCK_LARG': 'int', 'PRRCK_MDDL': 'int', 'CLZN_CD': 'int', 'TPGRP_TPCD': 'int', 'PRDN_FOM_C': 'int',
       'SLANT_TYP': 'int', 'SLDPT_TPCD': 'int', 'SCSTX_CD': 'int', 'SLTP_CD': 'int', 'STORUNST': 'int', 'FROR_CD': 'int',
       'FRTP_CD': 'int', 'KOFTR_GROU': 'int', 'DMCLS_CD': 'int', 'AGCLS_CD': 'int', 'HEIGHT': 'int'})

In [30]:
X_train_1 = X_train_1.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

In [31]:
X_test_1 = X_test_1.astype({'PRRCK_LARG': 'int', 'PRRCK_MDDL': 'int', 'CLZN_CD': 'int', 'TPGRP_TPCD': 'int', 'PRDN_FOM_C': 'int',
       'SLANT_TYP': 'int', 'SLDPT_TPCD': 'int', 'SCSTX_CD': 'int', 'SLTP_CD': 'int', 'STORUNST': 'int', 'FROR_CD': 'int',
       'FRTP_CD': 'int', 'KOFTR_GROU': 'int', 'DMCLS_CD': 'int', 'AGCLS_CD': 'int', 'HEIGHT': 'int'})

In [32]:
X_test_1 = X_test_1.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

In [ ]:
X_train_1

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT
729989,20.0,17.2,24.6,0.0,0.0,0.00,0.0,6.7,3.7,1.6,62.0,83.3,0.0,1,11,811.2,16.8,46.4,2,12,6,3,30,1,2,1,2,2,32,1,4,12
613783,20.9,16.1,28.3,0.0,0.0,0.00,0.0,7.0,3.2,1.2,51.0,84.8,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,2,1,11,2,4,12
157600,20.1,16.3,23.8,0.0,0.2,0.00,0.3,4.7,2.5,0.8,67.0,85.9,0.0,1,11,525.9,34.5,280.8,2,12,6,3,30,1,2,1,2,2,32,1,4,12
97378,28.8,25.2,34.6,0.0,0.0,0.00,0.0,7.1,4.1,1.4,45.0,72.4,0.0,1,16,502.2,30.3,335.3,3,12,1,1,10,2,1,1,2,1,11,2,5,12
32900,21.9,17.2,29.7,2.0,12.5,4.28,18.5,6.6,4.6,2.1,40.0,74.1,56.0,2,23,70.4,22.6,342.7,4,7,1,1,10,3,15,1,2,1,15,1,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646685,21.9,17.5,28.8,0.0,0.0,0.00,0.0,4.1,3.3,0.8,50.0,80.3,0.0,2,24,410.9,34.6,340.3,3,7,5,3,20,3,14,1,2,1,11,2,4,16
727116,23.0,21.5,26.5,5.0,15.5,0.00,38.5,6.2,3.4,0.3,86.0,96.1,0.0,1,11,811.2,16.8,46.4,2,12,6,3,30,1,2,1,2,2,32,1,4,12
30816,26.7,24.6,28.7,2.0,10.1,14.27,46.8,11.1,7.3,4.7,75.0,85.1,3.2,2,23,70.4,22.6,342.7,4,7,1,1,10,3,15,1,2,1,15,1,3,12
661272,20.3,16.6,24.8,0.0,0.0,0.00,0.0,7.3,4.7,2.2,67.0,83.6,0.0,2,22,626.9,36.7,123.9,2,12,1,1,10,3,13,1,1,1,12,1,3,14


In [ ]:
X_train_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13731 entries, 16778 to 6717
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   avgTa       13731 non-null  float64
 1   minTa       13731 non-null  float64
 2   maxTa       13731 non-null  float64
 3   mi10MaxRn   13731 non-null  float64
 4   hr1MaxRn    13731 non-null  float64
 5   sumRnDur    13731 non-null  float64
 6   sumRn       13731 non-null  float64
 7   maxInsWs    13731 non-null  float64
 8   maxWs       13731 non-null  float64
 9   avgWs       13731 non-null  float64
 10  minRhm      13731 non-null  float64
 11  avgRhm      13731 non-null  float64
 12  n99Rn       13731 non-null  float64
 13  PRRCK_LARG  13731 non-null  object 
 14  PRRCK_MDDL  13731 non-null  object 
 15  LOCTN_ALTT  13731 non-null  float64
 16  LOCTN_GRDN  13731 non-null  float64
 17  EIGHT_AGL   13731 non-null  float64
 18  CLZN_CD     13731 non-null  object 
 19  TPGRP_TPCD  13731 non-

In [ ]:
cat_features = np.where(X_train_1.dtypes.astype("str").isin(["category","object"]))[0]
cat_features

array([13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [60]:
from catboost import CatBoostClassifier

# cat_features = np.where(X_train_1.dtypes.astype("str").isin(["category","object"]))[0]
cat_features = [13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
# cat_features = ['PRRCK_LARG', 'PRRCK_MDDL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
#        'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
#        'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT']

model = CatBoostClassifier(learning_rate= iterations=20)

cat1 = model.fit(X_train_1, y_train_1, eval_set=(X_test_1, y_test_1),
            early_stopping_rounds=20, cat_features=cat_features)
pred_cat1 = model.predict(X_test_1)
metrics(y_test_1, pred_cat1)

Learning rate set to 0.5
0:	learn: 0.4034324	test: 0.4038963	best: 0.4038963 (0)	total: 81ms	remaining: 1.54s
1:	learn: 0.2633016	test: 0.2639457	best: 0.2639457 (1)	total: 147ms	remaining: 1.33s
2:	learn: 0.1841145	test: 0.1840665	best: 0.1840665 (2)	total: 212ms	remaining: 1.2s
3:	learn: 0.1496164	test: 0.1498595	best: 0.1498595 (3)	total: 303ms	remaining: 1.21s
4:	learn: 0.1247119	test: 0.1246774	best: 0.1246774 (4)	total: 367ms	remaining: 1.1s
5:	learn: 0.1052111	test: 0.1046250	best: 0.1046250 (5)	total: 428ms	remaining: 998ms
6:	learn: 0.0927528	test: 0.0919086	best: 0.0919086 (6)	total: 492ms	remaining: 914ms
7:	learn: 0.0771973	test: 0.0761322	best: 0.0761322 (7)	total: 570ms	remaining: 856ms
8:	learn: 0.0691847	test: 0.0681426	best: 0.0681426 (8)	total: 630ms	remaining: 770ms
9:	learn: 0.0598298	test: 0.0587263	best: 0.0587263 (9)	total: 697ms	remaining: 697ms
10:	learn: 0.0564035	test: 0.0552934	best: 0.0552934 (10)	total: 763ms	remaining: 624ms
11:	learn: 0.0503966	test: 0.0

# LightGBM

In [ ]:
from lightgbm import LGBMClassifier, plot_importance

lgb = LGBMClassifier(n_estimaotrs = 400,boost_from_average = False )
evals = [(X_test_1, y_test_1)]
lgb1 = lgb.fit(X_train_1, y_train_1, early_stopping_rounds = 1000, eval_metric = "logloss", eval_set = evals, verbose = True)

In [ ]:
lgb_pred = lgb1.predict(X_test_1)
metrics(y_test_1, lgb_pred)

정확도 : 0.93, 정밀도 : 0.88, 재현율 : 1.00
f1-score : 0.94, auc : 0.93


# scaling - 표준화 작업 
각 특성의 평균을 0, 분산을 1로 스케일링합니다.
즉, 데이터를 정규분포로 만듭니다.
최솟값과 최댓값의 크기를 제한하지 않기 때문에, 어떤 알고리즘에서는 문제가 있을 수 있습니다.
회귀보다 분류에 유용합니다.

In [ ]:
X_train_1.shape

(3140146, 26)

In [ ]:
#clean_df_drop = clean_df.drop(['tm','umd2','landslide','1day_yn','2day_yn'],axis=1)
#column = clean_df_drop.columns.tolist()
pd.set_option('display.max_rows', 500)

scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
scaler.fit(X_train_1)
X_train_1 = scaler.transform(X_train_1)

scaler.fit(X_test_1)
X_test_1 = scaler.transform(X_test_1)
#data_df_scaled = pd.DataFrame(data=data_scaled, columns = columns_select_day1)
#print('feature 들의 평균 값')
#print(data_df_scaled.mean())
#print('\nfeature 들의 분산 값')
#print(data_df_scaled.var())

In [ ]:
X_test_1.shape

(1046716, 26)

# Imbalace-xgboost

 모델을 돌리기전에 데이터 타입을 numpy 등으로 바꿔줘야함 - 샘플링을 하는 경우 자동으로 적용됨(참고)


In [70]:
pip install imbalance-xgboost

     |████████████████████████████████| 166.7 MB 3.8 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [71]:
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb

#xgboster_weight = imb_xgb(special_objective='weighted')
#CV_weight_booster1 = GridSearchCV(xgboster_weight,  {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
#CV_weight_booster1 = GridSearchCV(xgboster_weight,  {"imbalance_alpha":[1.5]})
#CV_weight_booster1.fit(X_train_1, y_train_1)
#opt_weight_booster1 = CV_weight_booster1.best_estimator_

#focal method
xgboster_focal = imb_xgb(special_objective='focal')
CV_focal_booster1 = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0]})
CV_focal_booster1.fit(X_train_1, y_train_1)
opt_focal_booster1 = CV_focal_booster1.best_estimator_
#metrics(y_test_1, CV_weight_booster1)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: "None of [Int64Index([463887, 187597,   2802, 129076,  83816, 275398, 475175, 160219,\n            262109, 101756,\n            ...\n            448359, 183328, 269345, 149295, 286276, 423824, 336474,  99076,\n             71295, 320081],\n           dtype='int64', length=361545)] are in the [columns]"

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
KeyError: "None of [Int64Index([322708, 253697, 460312, 424342, 287624, 239831, 358165, 257410,\n             27634, 409457,\n            ...\n            216563, 135949, 468442, 354861, 112093, 256897, 404386,  37023,\n            363739,

KeyError: ignored

In [ ]:
class_output = opt_focal_booster1.predict_determine(X_test_1, y=None)
class_output

array([1., 0., 1., ..., 1., 0., 0.], dtype=float32)

# XGBOOST

In [69]:
xgb = XGBClassifier()

#xgb_param_grid = {
#    'n_estimators' : [100, 200, 400, 600],
#    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
#    'max_depth' : [4, 6, 8, 10, 12],
#}
#xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
#xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="accuracy", n_jobs= -1, verbose = 1)
evals = [(X_test_1, y_test_1)]
xgb1 =xgb.fit(X_train_1, y_train_1, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

ValueError: ignored

In [ ]:
xgb_pred = xgb1.predict(X_test_1)
metrics(y_test_1, xgb_pred)

정확도 : 0.93, 정밀도 : 0.88, 재현율 : 1.00
f1-score : 0.94, auc : 0.93


In [ ]:
fig, ax = plt.subplots(figsize=(50,50))
plt.figure(figsize=(100,80))
plot_importance(xgb, ax=ax)

# 앙상블
#### bagging, bossting, stackin, voting의 방법중에서 voting을 선정함
#### Voting이란 서로 다른 알고리즘을 가진 분류기를 결합하는 방법이다.


In [ ]:
import pickle

# Save Model
with open('./voting_model.pkl', 'wb') as f:
    pickle.dump(voting_clf, f)

# Load Model
with open('./voting_model.pkl', 'rb') as f:
    voting_clf = pickle.load(f)

# 2일 후 예측

In [ ]:
clean_df["2day_yn"].value_counts()

0.00000    401713
1.00000       359
Name: 2day_yn, dtype: int64

In [ ]:
columns_select_day2 = ['avgRhm', 'maxTa','minTa','avgTa','maxInsWs','avgWs','LOCTN_ALTT','LOCTN_GRDN','minRhm','maxWs','sumRn','hr1MaxRn','sumRnDur','n99Rn','mi10MaxRn','PRRCK_LARG_2.0','AGCLS_CD_5.0','HEIGHT_14.0','AGCLS_CD_6.0','KOFTR_GROU_11.0','EIGHT_AGL_dir_N'] 

In [34]:
#xgboostd의 feature importance를 통해 선정된 변수
columns_select_day2= ['avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT'] 

In [35]:
y_2 = clean_df['2day_yn']
X_2 = clean_df[columns_select_day2]
#X_2 = clean_df.drop(['1day_yn','2day_yn','tm','umd2','stnId'], axis =1)

In [36]:
X = pd.DataFrame(X_2, columns=columns_select_day2)
y = pd.DataFrame(y_2, columns = ['2day_yn'])
df_2day = pd.concat([X,y],axis=1)
df_2day

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,2day_yn
0,13.6,12.7,14.9,0.0,0.5,6.67,2.0,10.6,7.0,3.1,81.0,95.5,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
1,13.4,11.8,16.5,0.0,0.1,0.67,0.1,5.8,3.7,1.3,74.0,92.5,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
2,14.6,12.0,18.7,NaN,NaN,NaN,NaN,13.3,8.5,5.9,48.0,88.1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
3,20.7,18.2,23.9,NaN,NaN,NaN,NaN,19.2,12.0,6.2,29.0,53.4,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
4,21.0,17.9,23.6,NaN,NaN,NaN,NaN,13.1,9.2,3.6,37.0,48.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402067,12.1,8.3,18.0,NaN,NaN,NaN,NaN,3.6,1.7,1.0,33.0,61.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0
402068,13.4,7.3,19.0,NaN,NaN,NaN,NaN,4.7,2.9,1.0,45.0,65.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0
402069,15.4,10.7,21.0,NaN,NaN,NaN,NaN,6.4,4.1,1.6,35.0,62.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0
402070,13.3,9.0,20.0,NaN,NaN,NaN,NaN,3.7,1.8,1.1,40.0,64.6,NaN,1.0,11.0,241.2,27.1,83.6,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,3.0,77.0,1.0,3.0,10.0,0.0


In [37]:
day0 = df_2day[df_2day['2day_yn'] == 0].shape[0]
day1 = df_2day[df_2day['2day_yn'] == 1].shape[0]

In [38]:
df2_2y = df_2day[df_2day['2day_yn'] == 1].copy()
df2_2y

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,2day_yn
893,16.9,14.5,19.6,1.0,3.8,5.72,8.3,8.8,5.1,2.5,50.0,69.1,37.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,1.0
2727,15.6,13.4,18.7,4.0,9.6,9.67,21.1,19.3,12.9,6.1,72.0,86.6,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,1.0
4104,17.4,13.8,21.5,4.0,12.6,NaN,26.5,9.5,5.9,3.5,84.0,93.6,NaN,2.0,22.0,626.9,36.7,123.9,2.0,12.0,1.0,1.0,10.0,3.0,13.0,1.0,1.0,1.0,12.0,1.0,3.0,14.0,1.0
5481,17.4,13.8,21.5,4.0,12.6,NaN,26.5,9.5,5.9,3.5,84.0,93.6,NaN,2.0,23.0,583.3,38.7,279.9,2.0,12.0,5.0,3.0,20.0,4.0,14.0,1.0,1.0,1.0,11.0,0.0,1.0,0.0,1.0
6858,17.4,13.8,21.5,4.0,12.6,NaN,26.5,9.5,5.9,3.5,84.0,93.6,NaN,1.0,11.0,100.0,15.5,50.8,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,1.0,11.0,1.0,4.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396827,20.8,16.9,25.4,NaN,NaN,NaN,NaN,7.5,3.0,1.7,45.0,67.3,NaN,1.0,11.0,501.3,30.4,30.3,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,1.0,15.0,2.0,5.0,12.0,1.0
398204,20.8,16.9,25.4,NaN,NaN,NaN,NaN,7.5,3.0,1.7,45.0,67.3,NaN,1.0,11.0,501.3,30.4,30.3,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,1.0,15.0,2.0,5.0,12.0,1.0
399581,20.8,16.9,25.4,NaN,NaN,NaN,NaN,7.5,3.0,1.7,45.0,67.3,NaN,1.0,11.0,334.9,25.8,118.1,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,2.0,34.0,2.0,4.0,8.0,1.0
400211,19.8,18.3,22.6,1.0,2.0,NaN,2.6,6.5,4.0,1.2,60.0,84.5,NaN,1.0,11.0,334.9,25.8,118.1,4.0,12.0,1.0,1.0,10.0,2.0,1.0,1.0,2.0,2.0,34.0,2.0,4.0,8.0,1.0


1118

In [39]:
for _ in range(day0//day1):
  df_2day = pd.concat([df_2day,df2_2y],axis=0)
  df_2day

In [40]:
df_2day[df_2day['2day_yn'] == 0].shape

(401713, 33)

In [41]:
df_2day[df_2day['2day_yn'] == 1].shape

(401721, 33)

In [42]:
X_2by1 = df_2day.drop(['2day_yn'],axis=1).reset_index(drop=True)
y_2by1 = df_2day['2day_yn'].reset_index(drop=True)

###  오버샘플링

In [ ]:
#SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_over_2, y_train_over_2 = smote.fit_sample(X_2, y_2)
X_train_over_2 = pd.DataFrame(X_train_over_2, columns=columns_select_day2)
y_train_over_2 = pd.DataFrame(y_train_over_2)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


### 언더 샘플링

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp_2, y_samp_2 = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X_2, y_2)
#X_samp_2, y_samp_2 = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 7180}).fit_sample(X_2, y_2)

X_samp_2 = pd.DataFrame(X_samp_2, columns=columns_select_day2)
y_samp_2 = pd.DataFrame(y_samp_2)

#df_samp = pd.concat([X_df, y_df], axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_samp_2 = X_samp_2.astype('float')
X_samp_2 = X_samp_2.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

### Train, test 분리

In [43]:
# 1:1 ratio
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2by1, y_2by1, test_size=0.25, shuffle=True, stratify=y_2by1, random_state=1024)

#normal case
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.25, shuffle=True, stratify= y_2, random_state=1024)

#undersampling 분리
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_samp_2, y_samp_2, test_size=0.25, shuffle=True, stratify=y_samp_2, random_state=0)

#over sampliung 분리
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train_over_2, y_train_over_2, test_size=0.25, shuffle=True, stratify=y_train_over_2, random_state=0)

print('X_train shape:', X_train_2.shape)
print('X_test shape:', X_test_2.shape)
print('y_train shape:', y_train_2.shape)
print('y_test shape:', y_test_2.shape)

X_train shape: (602575, 32)
X_test shape: (200859, 32)
y_train shape: (602575,)
y_test shape: (200859,)


# SVM -시도 중이나 이유를 찾지 못함

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm

# Generate some abnormal novel observations
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))

# fit the model
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(X_train_1)
y_pred_train = clf.predict(X_train_1)
y_pred_test = clf.predict(X_test_1)
y_pred_outliers = clf.predict(X_outliers)


In [ ]:
#clean_df_drop = clean_df.drop(['tm','umd2','landslide','1day_yn','2day_yn'],axis=1)
#column = clean_df_drop.columns.tolist()
pd.set_option('display.max_rows', 500)

scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
scaler.fit(X_train_2)
X_train_2 = scaler.transform(X_train_2)

scaler.fit(X_test_2)
X_test_2 = scaler.transform(X_test_2)

#data_df_scaled = pd.DataFrame(data=data_scaled, columns = columns_select_day1)
#print('feature 들의 평균 값')
#print(data_df_scaled.mean())
#print('\nfeature 들의 분산 값')
#print(data_df_scaled.var())

# CATBOOST

In [ ]:
pip install catboost

     |████████████████████████████████| 67.4 MB 29 kB/s 


# CATBOOST

In [44]:
X_train_2 = X_train_2.astype({'PRRCK_LARG': 'int', 'PRRCK_MDDL': 'int', 'CLZN_CD': 'int', 'TPGRP_TPCD': 'int', 'PRDN_FOM_C': 'int',
       'SLANT_TYP': 'int', 'SLDPT_TPCD': 'int', 'SCSTX_CD': 'int', 'SLTP_CD': 'int', 'STORUNST': 'int', 'FROR_CD': 'int',
       'FRTP_CD': 'int', 'KOFTR_GROU': 'int', 'DMCLS_CD': 'int', 'AGCLS_CD': 'int', 'HEIGHT': 'int'})
X_train_2 = X_train_2.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})
X_test_2 = X_test_2.astype({'PRRCK_LARG': 'int', 'PRRCK_MDDL': 'int', 'CLZN_CD': 'int', 'TPGRP_TPCD': 'int', 'PRDN_FOM_C': 'int',
       'SLANT_TYP': 'int', 'SLDPT_TPCD': 'int', 'SCSTX_CD': 'int', 'SLTP_CD': 'int', 'STORUNST': 'int', 'FROR_CD': 'int',
       'FRTP_CD': 'int', 'KOFTR_GROU': 'int', 'DMCLS_CD': 'int', 'AGCLS_CD': 'int', 'HEIGHT': 'int'})
X_test_2 = X_test_2.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

In [ ]:
X_train_2

,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT
729989,18.2,12.6,24.6,0.0,0.0,0.00,0.0,7.7,4.3,1.4,55.0,81.4,0.0,1,11,811.2,16.8,46.4,2,12,6,3,30,1,2,1,2,2,32,1,4,12
613783,19.9,17.0,26.0,0.0,0.0,0.00,0.0,11.1,6.7,2.6,63.0,83.4,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,2,1,11,2,4,12
157600,20.1,16.3,23.8,0.0,0.2,0.00,0.3,4.7,2.5,0.8,67.0,85.9,0.0,1,11,525.9,34.5,280.8,2,12,6,3,30,1,2,1,2,2,32,1,4,12
97378,28.8,25.2,34.6,0.0,0.0,0.00,0.0,7.1,4.1,1.4,45.0,72.4,0.0,1,16,502.2,30.3,335.3,3,12,1,1,10,2,1,1,2,1,11,2,5,12
32900,21.9,17.2,29.7,2.0,12.5,4.28,18.5,6.6,4.6,2.1,40.0,74.1,56.0,2,23,70.4,22.6,342.7,4,7,1,1,10,3,15,1,2,1,15,1,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646685,22.1,18.4,26.9,0.0,0.0,0.00,0.0,3.7,3.0,1.0,50.0,76.5,0.0,2,24,410.9,34.6,340.3,3,7,5,3,20,3,14,1,2,1,11,2,4,16
727116,24.7,22.1,29.2,3.0,4.0,0.00,9.0,5.3,3.0,0.6,73.0,93.9,0.0,1,11,811.2,16.8,46.4,2,12,6,3,30,1,2,1,2,2,32,1,4,12
30816,26.7,24.6,28.7,2.0,10.1,14.27,46.8,11.1,7.3,4.7,75.0,85.1,3.2,2,23,70.4,22.6,342.7,4,7,1,1,10,3,15,1,2,1,15,1,3,12
661272,17.4,13.8,21.5,4.0,12.6,0.00,26.5,9.5,5.9,3.5,84.0,93.6,0.0,2,22,626.9,36.7,123.9,2,12,1,1,10,3,13,1,1,1,12,1,3,14


In [61]:
from catboost import CatBoostClassifier

cat_features = np.where(X_train_2.dtypes.astype("str").isin(["category","object"]))[0]
# cat_features = [13, 14, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
# cat_features = ['PRRCK_LARG', 'PRRCK_MDDL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
#        'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
#        'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT']

model = CatBoostClassifier(iterations=20)

cat2 = model.fit(X_train_2, y_train_2, eval_set=(X_test_2, y_test_2),
            early_stopping_rounds=20, cat_features=cat_features)
pred_cat2 = model.predict(X_test_2)
metrics(y_test_2,pred_cat2)

Learning rate set to 0.5
0:	learn: 0.4034348	test: 0.4048494	best: 0.4048494 (0)	total: 88.7ms	remaining: 1.69s
1:	learn: 0.3103716	test: 0.3118376	best: 0.3118376 (1)	total: 171ms	remaining: 1.54s
2:	learn: 0.2565527	test: 0.2585600	best: 0.2585600 (2)	total: 249ms	remaining: 1.41s
3:	learn: 0.2257785	test: 0.2270904	best: 0.2270904 (3)	total: 344ms	remaining: 1.38s
4:	learn: 0.1896234	test: 0.1908696	best: 0.1908696 (4)	total: 421ms	remaining: 1.26s
5:	learn: 0.1573767	test: 0.1583359	best: 0.1583359 (5)	total: 493ms	remaining: 1.15s
6:	learn: 0.1236178	test: 0.1241158	best: 0.1241158 (6)	total: 574ms	remaining: 1.07s
7:	learn: 0.1043452	test: 0.1046100	best: 0.1046100 (7)	total: 671ms	remaining: 1.01s
8:	learn: 0.0861362	test: 0.0861881	best: 0.0861881 (8)	total: 744ms	remaining: 909ms
9:	learn: 0.0782424	test: 0.0782399	best: 0.0782399 (9)	total: 825ms	remaining: 825ms
10:	learn: 0.0723534	test: 0.0722994	best: 0.0722994 (10)	total: 904ms	remaining: 739ms
11:	learn: 0.0675420	test:

# LightGBM

In [ ]:
from lightgbm import LGBMClassifier, plot_importance

lgb = LGBMClassifier(n_estimaotrs = 400)
evals = [(X_test_2, y_test_2)]
lgb2 = lgb.fit(X_train_2, y_train_2, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = True)

In [ ]:
lgb_pred_2 = lgb2.predict(X_test_2)
metrics(y_test_2, lgb_pred_2)

정확도 : 0.91, 정밀도 : 0.85, 재현율 : 0.98
f1-score : 0.92, auc : 0.91


# Imbalace-xgboost

 모델을 돌리기전에 데이터 타입을 numpy 등으로 바꿔줘야함 - 샘플링을 하는 경우 자동으로 적용됨(참고)


In [ ]:
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb

xgboster_weight = imb_xgb(special_objective='weighted')
CV_weight_booster2 = GridSearchCV(xgboster_weight,  {"imbalance_alpha":[1.5]})
#CV_weight_booster2 = GridSearchCV(xgboster_weight,  {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
CV_weight_booster2.fit(X_train_2, y_train_2)
opt_weight_booster2 = CV_weight_booster2.best_estimator_

#focal method
#xgboster_focal = imb_xgb(special_objective='focal')
#CV_focal_booster2 = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0]})
#CV_focal_booster2.fit(X_train_2, y_train_2)
#opt_focal_booster2 = CV_focal_booster2.best_estimator_


# XGboost

In [ ]:
xgb = XGBClassifier()

#xgb_param_grid = {
#    'n_estimators' : [100, 200, 400, 600],
#    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
#    'max_depth' : [4, 6, 8, 10, 12],
#}
#xgb2 = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
#xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="accuracy", n_jobs= -1, verbose = 1)
evals = [(X_test_2, y_test_2)]
xgb2 = xgb2.fit(X_train_2, y_train_2, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-logloss:0.63657
[1]	validation_0-logloss:0.58997
[2]	validation_0-logloss:0.55122
[3]	validation_0-logloss:0.51970
[4]	validation_0-logloss:0.48765
[5]	validation_0-logloss:0.46079
[6]	validation_0-logloss:0.43832
[7]	validation_0-logloss:0.41503
[8]	validation_0-logloss:0.39448
[9]	validation_0-logloss:0.37717
[10]	validation_0-logloss:0.35898
[11]	validation_0-logloss:0.34529
[12]	validation_0-logloss:0.33477
[13]	validation_0-logloss:0.32180
[14]	validation_0-logloss:0.30918
[15]	validation_0-logloss:0.30442
[16]	validation_0-logloss:0.30041
[17]	validation_0-logloss:0.29242
[18]	validation_0-logloss:0.28898
[19]	validation_0-logloss:0.28438
[20]	validation_0-logloss:0.27810
[21]	validation_0-logloss:0.27678
[22]	validation_0-logloss:0.27636
[23]	validation_0-logloss:0.27750
[24]	validation_0-logloss:0.27963
[25]	validation_0-logloss:0.27697
[26]	validation_0-logloss:0.27354
[27]	validation_0-logloss:0.27118
[28]	validation_0-logloss:0.27070
[29]	validation_0-loglos

In [ ]:
xgb_pred_2 = xgb2.predict(X_test_2)
metrics(y_test_2, xgb_pred_2)

정확도 : 0.88, 정밀도 : 0.81, 재현율 : 1.00
f1-score : 0.89, auc : 0.88


In [ ]:
fig, ax = plt.subplots(figsize=(100,80))
plt.figure(figsize=(100,80))
plot_importance(xgb2, ax=ax)


# Submission 실행

In [46]:
# Test 검증 데이터셋
test_input = pd.read_csv('test_final.csv', encoding = 'utf-8')
#test_input = pd.read_csv('data/210039_한글.csv', encoding = 'utf-8')
#불필요한 칼럼제거
X_test = test_input.drop(['1day_yn','2day_yn','date','sd','sgg','umd_x','1day','2day','tm','stnId','stnNm','umd_y','umd2'], axis =1)
#columns = X_test.columns.tolist()

X_test = X_test.fillna(0)
X_test = X_test.astype({'PRRCK_LARG': 'int', 'PRRCK_MDDL': 'int', 'CLZN_CD': 'int', 'TPGRP_TPCD': 'int', 'PRDN_FOM_C': 'int',
       'SLANT_TYP': 'int', 'SLDPT_TPCD': 'int', 'SCSTX_CD': 'int', 'SLTP_CD': 'int', 'STORUNST': 'int', 'FROR_CD': 'int',
       'FRTP_CD': 'int', 'KOFTR_GROU': 'int', 'DMCLS_CD': 'int', 'AGCLS_CD': 'int', 'HEIGHT': 'int'})
X_test = X_test.astype({'PRRCK_LARG': 'object', 'PRRCK_MDDL': 'object', 'CLZN_CD': 'object', 'TPGRP_TPCD': 'object', 'PRDN_FOM_C': 'object',
       'SLANT_TYP': 'object', 'SLDPT_TPCD': 'object', 'SCSTX_CD': 'object', 'SLTP_CD': 'object', 'STORUNST': 'object', 'FROR_CD': 'object',
       'FRTP_CD': 'object', 'KOFTR_GROU': 'object', 'DMCLS_CD': 'object', 'AGCLS_CD': 'object', 'HEIGHT': 'object'})

#xgboost 결과 중요칼럼 추출 (날짜별)
#1day
columns1 = ['avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT'] 
X_test_day1 = X_test[columns_select_day1]

#2day
columns2 =['avgTa', 'minTa', 'maxTa', 'mi10MaxRn',
       'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs', 'minRhm',
       'avgRhm', 'n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL', 'LOCTN_ALTT',
       'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD', 'PRDN_FOM_C',
       'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD', 'STORUNST', 'FROR_CD',
       'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD', 'HEIGHT'] 
X_test_day2 = X_test[columns_select_day2]


In [ ]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3410 entries, 0 to 3409
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   avgTa       3410 non-null   float64
 1   minTa       3410 non-null   float64
 2   maxTa       3410 non-null   float64
 3   mi10MaxRn   3410 non-null   float64
 4   hr1MaxRn    3410 non-null   float64
 5   sumRnDur    3410 non-null   float64
 6   sumRn       3410 non-null   float64
 7   maxInsWs    3410 non-null   float64
 8   maxWs       3410 non-null   float64
 9   avgWs       3410 non-null   float64
 10  minRhm      3410 non-null   float64
 11  avgRhm      3410 non-null   float64
 12  n99Rn       3410 non-null   float64
 13  PRRCK_LARG  3410 non-null   object 
 14  PRRCK_MDDL  3410 non-null   object 
 15  LOCTN_ALTT  3410 non-null   float64
 16  LOCTN_GRDN  3410 non-null   float64
 17  EIGHT_AGL   3410 non-null   float64
 18  CLZN_CD     3410 non-null   object 
 19  TPGRP_TPCD  3410 non-null  

In [ ]:
#clean_df_drop = clean_df.drop(['tm','umd2','landslide','1day_yn','2day_yn'],axis=1)
#column = clean_df_drop.columns.tolist()
pd.set_option('display.max_rows', 500)

scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
scaler.fit(X_test_day1)
X_test_day1 = scaler.transform(X_test_day1)
scaler.fit(X_test_day2)
X_test_day2 = scaler.transform(X_test_day2)
#data_df_scaled = pd.DataFrame(data=data_scaled, columns = columns_select_day1)
#print('feature 들의 평균 값')
#print(data_df_scaled.mean())
#print('\nfeature 들의 분산 값')
#print(data_df_scaled.var())

In [62]:
#catboost
#1day 예측
pred_test_day1 = cat1.predict(X_test_day1)
#2day 예측
pred_test_day2 = cat2.predict(X_test_day2)

In [ ]:
#lgbboost
#1day 예측
pred_test_day1 = lgb1.predict(X_test_day1)
#2day 예측
pred_test_day2 = lgb2.predict(X_test_day2.to_numpy())

ValueError: ignored

In [ ]:
#imxgb
#weight 방법
#1day 예측
pred_test_day1 = opt_weight_booster1.predict_determine(X_test_day1)
#2day 예측
pred_test_day2 = opt_weight_booster2.predict_determine(X_test_day2)

#focal 방법
#1day 예측
#pred_test_day1 = opt_focal_booster1.predict_determine(X_test_day1.to_numpy())
#2day 예측
#pred_test_day2 = opt_focal_booster2.predict_determine(X_test_day2.to_numpy())


NameError: ignored

In [ ]:
pred_test_day1

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
#xgboost
#1day 예측
pred_test_day1 = xgb1.predict(X_test_day1)
#2day 예측
pred_test_day2 = xgb2.predict(X_test_day2)

NameError: ignored

In [63]:
#제출 파일 생성
test_input['1day_yn'] = pred_test_day1
test_input['2day_yn'] = pred_test_day2
test_input.head()

,date,sd,sgg,umd_x,1day,1day_yn,2day,2day_yn,tm,stnId,stnNm,umd_y,umd2,avgTa,minTa,maxTa,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT
0,2020-06-10,경상남도,사천시,곤명면,2020-06-11,0.0,2020-06-12,0.0,2020-06-10,192,진주,곤명면,사천시 곤명면,24.7,18.3,32.8,0.0,0.4,0.0,0.4,7.6,3.2,0.9,48.0,75.8,3.6,2.0,22.0,87.2,18.7,245.2,4.0,7.0,1.0,1.0,10.0,2.0,26.0,1,2,1,11,1,4,10.0
1,2020-06-11,경상남도,사천시,곤명면,2020-06-12,0.0,2020-06-13,0.0,2020-06-11,192,진주,곤명면,사천시 곤명면,24.8,20.7,31.2,0.0,1.4,0.0,3.2,5.8,2.5,0.8,62.0,85.9,0.0,2.0,22.0,87.2,18.7,245.2,4.0,7.0,1.0,1.0,10.0,2.0,26.0,1,2,1,11,1,4,10.0
2,2020-06-12,경상남도,사천시,곤명면,2020-06-13,0.0,2020-06-14,0.0,2020-06-12,192,진주,곤명면,사천시 곤명면,24.6,21.7,30.2,4.0,11.5,0.0,11.2,6.8,3.0,0.9,67.0,87.5,48.7,2.0,22.0,87.2,18.7,245.2,4.0,7.0,1.0,1.0,10.0,2.0,26.0,1,2,1,11,1,4,10.0
3,2020-07-10,경상남도,사천시,곤명면,2020-07-11,0.0,2020-07-12,0.0,2020-07-10,192,진주,곤명면,사천시 곤명면,23.9,21.7,28.7,5.0,15.6,0.0,62.6,4.9,2.8,1.0,72.0,91.6,3.7,2.0,22.0,87.2,18.7,245.2,4.0,7.0,1.0,1.0,10.0,2.0,26.0,1,2,1,11,1,4,10.0
4,2020-07-11,경상남도,사천시,곤명면,2020-07-12,0.0,2020-07-13,0.0,2020-07-11,192,진주,곤명면,사천시 곤명면,24.7,21.7,29.7,0.0,0.0,0.0,0.0,5.0,2.9,1.1,70.0,87.3,0.0,2.0,22.0,87.2,18.7,245.2,4.0,7.0,1.0,1.0,10.0,2.0,26.0,1,2,1,11,1,4,10.0


In [64]:
test_input["1day_yn"].value_counts()

0.0    3398
1.0      12
Name: 1day_yn, dtype: int64

In [65]:
test_input["2day_yn"].value_counts()

0.0    3388
1.0      22
Name: 2day_yn, dtype: int64

In [ ]:
# 파일 저장
test_input.to_csv("210039_cat1by1null.csv",encoding='euc-kr',index=False)

In [66]:
final = test_input[['date',	'sd',	'sgg',	'umd_x',	'1day',	'1day_yn',	'2day',	'2day_yn']]

In [67]:
final = final.rename(columns={'umd_x':'umd'})

In [68]:
final.to_csv("210039_cat1by1null.csv",encoding='euc-kr',index=False)